In [1]:
# -*- coding:utf-8 -*-
# Modified Author: Inyong Hwang (inyong1020@gmail.com)
# Date: 2019-08-06-Tue
# 파이토치 첫걸음 Chapter 4. 이미지 처리와 합성곱 신경망

import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

# 4.3 전이 학습

# 4.3.1 데이터 준비

from torchvision.datasets import ImageFolder
from torchvision import transforms

train_imgs = ImageFolder('./taco_and_burrito/train/',
                        transform=transforms.Compose([transforms.RandomCrop(224), transforms.ToTensor()]))
test_imgs = ImageFolder('./taco_and_burrito/test/',
                        transform=transforms.Compose([transforms.CenterCrop(224), transforms.ToTensor()]))

train_loader = DataLoader(train_imgs, batch_size=32, shuffle=True)
test_loader = DataLoader(test_imgs, batch_size=32, shuffle=False)

In [2]:
print(train_imgs.classes)

['burrito', 'taco']


In [3]:
print(train_imgs.class_to_idx)

{'burrito': 0, 'taco': 1}


In [4]:
# 4.3.2 파이토치를 사용한 전이 학습

from torchvision import models

net = models.resnet18(pretrained=True)

for p in net.parameters():
    p.requires_grad=False

fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)

In [5]:
def eval_net(net, data_loader, device="cpu"):
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        x = x.to(device)
        y = y.to(device)
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
        
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

def train_net(net, train_loader, test_loader, only_fc=True,
              optimizer_cls=optim.Adam,
              loss_fn=nn.CrossEntropyLoss(),
              n_iter=10, device="cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    if only_fc:
        optimizer = optimizer_cls(net.fc.parameters())
    else:
        optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        net.train()
        n = 0
        n_acc = 0
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        train_acc.append(n_acc / n)

        val_acc.append(eval_net(net, test_loader, device))
        print(epoch, train_losses[-1], train_acc[-1], val_acc[-1], flush=True)

In [6]:
net.to("cuda:0")

train_net(net, train_loader, test_loader, n_iter=20, device="cuda:0")

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:03<00:00,  7.04it/s]


0 0.7419205958192999 0.5926966292134831 0.6500000357627869


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:02<00:00,  9.90it/s]


1 0.5739482505754991 0.7247191011235955 0.8166667222976685


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:02<00:00, 10.24it/s]


...

17 0.298419480974024 0.8764044943820225 0.8833333849906921


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:02<00:00, 10.18it/s]


18 0.3105696819045327 0.8862359550561798 0.8666667342185974


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:02<00:00, 10.20it/s]


19 0.30700918659567833 0.8778089887640449 0.8833333849906921
